In [9]:
class UnionFind:
    """Union-Find (Disjoint Set Union) データ構造"""
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        """要素xの根を見つける（経路圧縮付き）"""
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        """要素xとyを結合する"""
        px, py = self.find(x), self.find(y)
        if px == py:
            return False  # 既に同じ集合
        
        # ランクによる結合最適化
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        return True

def parse_graph_input(input_text):
    """
    入力テキストからグラフデータを解析
    入力形式: "始点ノード 終点ノード 重み" の行の集合
    """
    edges = []
    lines = input_text.strip().split('\n')
    
    for line in lines:
        if line.strip():
            parts = line.strip().split()
            if len(parts) >= 3:
                u, v, w = int(parts[0]), int(parts[1]), int(parts[2])
                edges.append((u, v, w))
    
    return edges

def remove_duplicate_edges(edges):
    """無向グラフの重複エッジを除去"""
    edge_dict = {}
    
    for u, v, w in edges:
        # エッジを正規化（小さい方を先に）
        key = (min(u, v), max(u, v))
        
        # 既存のエッジがない場合、またはより小さい重みの場合に更新
        if key not in edge_dict or w < edge_dict[key]:
            edge_dict[key] = w
    
    # 辞書から元の形式に戻す
    unique_edges = []
    for (u, v), w in edge_dict.items():
        unique_edges.append((u, v, w))
    
    return unique_edges

def kruskal_mst(edges, verbose=True):
    """
    Kruskalのアルゴリズムで最小全域木を求める
    
    Args:
        edges: エッジのリスト [(u, v, weight), ...]
        verbose: 実行過程を表示するかどうか
    
    Returns:
        mst: 最小全域木のエッジリスト
        total_weight: 最小全域木の総重み
    """
    
    # 重複エッジを除去
    edges = remove_duplicate_edges(edges)
    
    # エッジを重みでソート
    edges.sort(key=lambda x: x[2])
    
    
    # ノード数を確認
    nodes = set()
    for u, v, w in edges:
        nodes.add(u)
        nodes.add(v)
    
    if not nodes:
        return [], 0
    
    max_node = max(nodes)
    uf = UnionFind(max_node + 1)
    mst = []
    total_weight = 0
    
    if verbose:
        print(f"\nアルゴリズム実行過程:")
    
    for u, v, w in edges:
        if uf.union(u, v):
            mst.append((u, v, w))
            total_weight += w
            if verbose:
                print(f"ステップ {len(mst):2d}: エッジ ({u}, {v}) 重み={w} を追加")
        else:
            if verbose:
                print(f"スキップ   : エッジ ({u}, {v}) 重み={w} (サイクルを作るため)")
        
        # n-1個のエッジが見つかったら終了（nはノード数）
        if len(mst) == len(nodes) - 1:
            break
    
    return mst, total_weight

def print_result(mst, total_weight):
    """結果を整形して表示"""
    print(f"\n=== 最小全域木の結果 ===")
    print(f"最小全域木の構成:")
    for i, (u, v, w) in enumerate(mst, 1):
        print(f"{i:2d}: ({u}, {v}) 重み={w}")
    
    print(f"\n最小全域木の総重み: {total_weight}")
    print(f"エッジ数: {len(mst)}")

def solve_graph(graph_data):
    """
    文字列からグラフを解析して最小全域木を求める関数
    """
    
    edges = parse_graph_input(graph_data)
    
    if not edges:
        print("有効なエッジが見つかりませんでした。")
        return [], 0
    
    print(f"入力されたエッジ数: {len(edges)}")
    
    # 最小全域木を求める
    mst, total_weight = kruskal_mst(edges, verbose=True)
    
    # 結果を表示
    print_result(mst, total_weight)
    
    return mst, total_weight

if __name__ == "__main__":
    # ここに入力データを書く
    graph_input = """
1 2 8
1 3 1
1 4 3
1 5 2
2 1 8
2 5 3
3 1 1
3 4 2
3 6 7
4 1 3
4 3 2
4 5 5
4 7 1
5 1 2
5 2 3
5 4 5
5 7 4
5 8 6
6 3 7
6 7 8
7 4 1
7 5 4
7 6 8
7 8 5
8 5 6
8 7 5
"""
    
    # 最小全域木を求める
    solve_graph(graph_input)

入力されたエッジ数: 26

アルゴリズム実行過程:
ステップ  1: エッジ (1, 3) 重み=1 を追加
ステップ  2: エッジ (4, 7) 重み=1 を追加
ステップ  3: エッジ (1, 5) 重み=2 を追加
ステップ  4: エッジ (3, 4) 重み=2 を追加
スキップ   : エッジ (1, 4) 重み=3 (サイクルを作るため)
ステップ  5: エッジ (2, 5) 重み=3 を追加
スキップ   : エッジ (5, 7) 重み=4 (サイクルを作るため)
スキップ   : エッジ (4, 5) 重み=5 (サイクルを作るため)
ステップ  6: エッジ (7, 8) 重み=5 を追加
スキップ   : エッジ (5, 8) 重み=6 (サイクルを作るため)
ステップ  7: エッジ (3, 6) 重み=7 を追加

=== 最小全域木の結果 ===
最小全域木の構成:
 1: (1, 3) 重み=1
 2: (4, 7) 重み=1
 3: (1, 5) 重み=2
 4: (3, 4) 重み=2
 5: (2, 5) 重み=3
 6: (7, 8) 重み=5
 7: (3, 6) 重み=7

最小全域木の総重み: 21
エッジ数: 7
